In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catboost

In [ ]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold, train_test_split
from sklearn.metrics import f1_score 

from catboost import Pool,CatBoostClassifier


In [ ]:
#데이터 불러오기
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
d_code=pd.read_csv('/content/속성_D_코드.csv',index_col=0).T.to_dict()
h_code=pd.read_csv('/content/속성_H_코드.csv',index_col=0).T.to_dict()
l_code=pd.read_csv('/content/속성_L_코드.csv',index_col=0).T.to_dict()

In [ ]:
#데이터 재가공
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df


train = add_code(train, d_code, h_code, l_code)
test = add_code(test, d_code, h_code, l_code)

In [ ]:
train = train.drop(['id', 'contents_open_dt','person_rn', 'contents_rn',
# 'person_prefer_d_1','person_prefer_d_2','person_prefer_d_3',
# 'contents_attribute_d','person_prefer_h_1','person_prefer_h_2','person_prefer_h_3','contents_attribute_h','contents_attribute_l',
'person_prefer_f','person_prefer_g'], axis=1) 


test = test.drop(['id', 'contents_open_dt','person_rn', 'contents_rn',
# 'person_prefer_d_1','person_prefer_d_2','person_prefer_d_3',
# 'contents_attribute_d','person_prefer_h_1','person_prefer_h_2','person_prefer_h_3','contents_attribute_h','contents_attribute_l',
'person_prefer_f','person_prefer_g'], axis=1)

# print(train.head())
train['d_l_match_yn']=train['d_l_match_yn'].replace([True,False],[1,0])
train['d_m_match_yn']=train['d_m_match_yn'].replace([True,False],[1,0])
train['d_s_match_yn']=train['d_s_match_yn'].replace([True,False],[1,0])
train['h_l_match_yn']=train['h_l_match_yn'].replace([True,False],[1,0])
train['h_m_match_yn']=train['h_m_match_yn'].replace([True,False],[1,0])
train['h_s_match_yn']=train['h_s_match_yn'].replace([True,False],[1,0])

test['d_l_match_yn']=test['d_l_match_yn'].replace([True,False],[1,0])
test['d_m_match_yn']=test['d_m_match_yn'].replace([True,False],[1,0])
test['d_s_match_yn']=test['d_s_match_yn'].replace([True,False],[1,0])
test['h_l_match_yn']=test['h_l_match_yn'].replace([True,False],[1,0])
test['h_m_match_yn']=test['h_m_match_yn'].replace([True,False],[1,0])
test['h_s_match_yn']=test['h_s_match_yn'].replace([True,False],[1,0])

In [ ]:
# def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:
#     df = df.copy()
#     df_code = df_code.copy()
#     df_code = df_code.add_prefix(f"{col}_")
#     #print(df_code)
#     df_code.columns.values[0] = col
#     #print(df_code)
#     return pd.merge(df,df_code,how="left",on=col)

In [ ]:
# def preprocess_data(
#                     df:pd.DataFrame,is_train:bool = True, cols_merge:List[Tuple[str,pd.DataFrame]] = []  , cols_equi:List[Tuple[str,str]]= [] ,
#                     cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
#                     )->Tuple[pd.DataFrame,np.ndarray]:
#     df = df.copy()

#     y_data = None
#     if is_train:
#         y_data = df["target"].to_numpy()
#         df = df.drop(columns="target")

#     for col, df_code in cols_merge:
#         df = merge_codes(df,df_code,col)

#     cols = df.select_dtypes(bool).columns.tolist()
#     df[cols] = df[cols].astype(int)

#     for col1, col2 in cols_equi:
#         df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

#     df = df.drop(columns=cols_drop)
#     return (df , y_data)

In [ ]:
# # 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
# cols_merge = [
#               ("person_prefer_d_1" , code_d),
#               ("person_prefer_d_2" , code_d),
#               ("person_prefer_d_3" , code_d),
#               ("contents_attribute_d" , code_d),
#               ("person_prefer_h_1" , code_h),
#               ("person_prefer_h_2" , code_h),
#               ("person_prefer_h_3" , code_h),
#               ("contents_attribute_h" , code_h),
#               ("contents_attribute_l" , code_l),
# ]

# # 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
# cols_equi = [

#     ("contents_attribute_c","person_prefer_c"),
#     ("contents_attribute_e","person_prefer_e"),

#     ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
#     ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
#     ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
#     ("person_prefer_d_2_attribute_d_l" , "contents_attribute_d_attribute_d_l"),
#     ("person_prefer_d_3_attribute_d_l" , "contents_attribute_d_attribute_d_s"),
#     ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_l"),


# ]

# # 학습에 필요없는 컬럼 리스트
# cols_drop = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt", "contents_rn", ]

In [ ]:
#컬럼 순서 변경
train.columns
train=train[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_e',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_m',
       'contents_attribute_e', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l','target']]

test=test[['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_e',
       'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_m',
       'contents_attribute_e', 'person_prefer_d_1_n',
       'person_prefer_d_1_s', 'person_prefer_d_1_m', 'person_prefer_d_1_l',
       'person_prefer_d_2_n', 'person_prefer_d_2_s', 'person_prefer_d_2_m',
       'person_prefer_d_2_l', 'person_prefer_d_3_n', 'person_prefer_d_3_s',
       'person_prefer_d_3_m', 'person_prefer_d_3_l', 'contents_attribute_d_n',
       'contents_attribute_d_s', 'contents_attribute_d_m',
       'contents_attribute_d_l', 'person_prefer_h_1_m', 'person_prefer_h_2_m',
       'person_prefer_h_3_m', 'contents_attribute_h_m', 'person_prefer_h_1_l',
       'person_prefer_h_2_l', 'person_prefer_h_3_l', 'contents_attribute_h_l',
       'contents_attribute_l_n', 'contents_attribute_l_s',
       'contents_attribute_l_m', 'contents_attribute_l_l']]


In [ ]:

x = train.iloc[:, :-1]
y = train.iloc[:, -1]

# x= x.to_numpy()

# x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)







# x_test=test.iloc[:]
# x_test=x_test.to_numpy()




In [ ]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()

In [ ]:
x_train.nunique()

d_l_match_yn                2
d_m_match_yn                2
d_s_match_yn                2
h_l_match_yn                2
h_m_match_yn                2
h_s_match_yn                2
person_attribute_a          2
person_attribute_a_1        8
person_attribute_b          6
person_prefer_c             5
person_prefer_e            12
contents_attribute_i        3
contents_attribute_a        3
contents_attribute_j_1      9
contents_attribute_j        2
contents_attribute_c        4
contents_attribute_k        2
contents_attribute_m        5
contents_attribute_e       12
person_prefer_d_1_n       442
person_prefer_d_1_s       137
person_prefer_d_1_m        36
person_prefer_d_1_l        11
person_prefer_d_2_n       430
person_prefer_d_2_s       136
person_prefer_d_2_m        36
person_prefer_d_2_l        11
person_prefer_d_3_n       418
person_prefer_d_3_s       136
person_prefer_d_3_m        36
person_prefer_d_3_l        11
contents_attribute_d_n    431
contents_attribute_d_s    137
contents_a

In [ ]:
is_holdout = False
n_splits = 5
iterations = 3000
patience = 50

cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
x_train.iloc[0:4]

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,person_prefer_e,contents_attribute_i,contents_attribute_a,contents_attribute_j_1,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_m,contents_attribute_e,person_prefer_d_1_n,person_prefer_d_1_s,person_prefer_d_1_m,person_prefer_d_1_l,person_prefer_d_2_n,person_prefer_d_2_s,person_prefer_d_2_m,person_prefer_d_2_l,person_prefer_d_3_n,person_prefer_d_3_s,person_prefer_d_3_m,person_prefer_d_3_l,contents_attribute_d_n,contents_attribute_d_s,contents_attribute_d_m,contents_attribute_d_l,person_prefer_h_1_m,person_prefer_h_2_m,person_prefer_h_3_m,contents_attribute_h_m,person_prefer_h_1_l,person_prefer_h_2_l,person_prefer_h_3_l,contents_attribute_h_l,contents_attribute_l_n,contents_attribute_l_s,contents_attribute_l_m,contents_attribute_l_l
209550,0,0,0,0,0,0,2,0,4,5,0,1,3,3,1,1,2,4,3,709,708,708,618,709,708,708,618,709,708,708,618,514,509,482,482,365,528,366,528,48,250,48,250,1834,1833,1832,2020
26567,0,0,0,1,1,1,2,0,3,5,4,2,3,5,1,1,1,1,4,452,451,450,377,1227,1227,1227,926,175,152,56,1,1056,1054,1053,926,568,566,569,568,308,308,312,308,981,980,954,2009
89164,1,1,1,0,0,0,1,0,4,1,5,3,1,5,1,1,2,1,6,782,782,745,744,819,815,745,744,38,37,2,1,782,782,745,744,316,398,432,543,3,94,149,277,1082,1060,1021,2010
428760,0,0,0,1,1,1,2,0,4,5,3,3,3,5,1,1,2,4,3,446,446,439,377,1195,1194,1193,926,110,109,56,1,114,109,56,1,381,380,382,381,71,71,71,71,1331,1331,1323,2011


In [ ]:
scores = []
models = []


for tri, vai in cv.split(x_train):
  print(x_train)
  print(tri)
  print(vai)
  print(len(x_train))
  print(len(tri))
  print(len(vai))
  break

  print("="*50)
  preds = []

  model = CatBoostClassifier(iterations=iterations,random_state=42,task_type="GPU",eval_metric="F1",cat_features=cat_features,one_hot_max_size=4)
  model.fit(x_train.iloc[tri], y_train[tri], 
          eval_set=[(x_train.iloc[vai], y_train[vai])], 
          early_stopping_rounds=patience ,
          verbose = 100
      )
  
  models.append(model)
  scores.append(model.get_best_score()["validation"]["F1"])
  if is_holdout:
      break    

        d_l_match_yn  ...  contents_attribute_l_l
209550             0  ...                    2020
26567              0  ...                    2009
89164              1  ...                    2010
428760             0  ...                    2011
21865              0  ...                    2009
...              ...  ...                     ...
251110             1  ...                    2017
184415             1  ...                    2020
390841             1  ...                    2006
439197             1  ...                    2009
167794             0  ...                    2022

[401560 rows x 47 columns]
[     0      1      3 ... 401555 401556 401558]
[     2      7     11 ... 401534 401557 401559]
401560
321248
80312


In [ ]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print(train)
    print(test)
    
    print("%s %s" % (train, test))
    print('==========================')

[2 3]
[0 1]
[2 3] [0 1]
[0 1]
[2 3]
[0 1] [2 3]


In [ ]:
print(scores)
print(np.mean(scores))

[]
nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
threshold = 0.4

In [ ]:
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

IndexError: ignored

In [ ]:
pred = np.mean( pred_list , axis = 0 )
pred = np.where(pred >= threshold , 1, 0)

In [ ]:
sample_submission = pd.read_csv('/content/sample_submission.csv')
sample_submission['target'] = pred
sample_submission

In [ ]:
sample_submission.to_csv("prediction.csv", index=False)